# Load and write daily AP RACMO

J. Kingslake April 17th, 2023.

In [2]:
import xarray as xr
import gcsfs
import fsspec
import json
with open('../secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)

## 1. Load the netcdfs 
Load one of the netcdf mentioned here: https://github.com/ldeo-glaciology/AntPen_NSF_NERC/issues/1#issuecomment-1512335716

I chose the t2m dataset.

In [3]:
gcs = gcsfs.GCSFileSystem()
url = 'gs://ldeo-glaciology/RACMO/RACMO_daily_by_var/RACMO_daily_t2m.nc'
of = gcs.open(url, mode='rb')
ds = xr.open_dataset(of,chunks={'time':300}) 


/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 300. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [4]:
ds.nbytes/1e9

13.517262784

In [7]:
ds.time[0], ds.time[-1]

(<xarray.DataArray 'time' ()>
 array('1979-01-01T00:00:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time     datetime64[ns] 1979-01-01
 Attributes:
     axis:       T
     long_name:  time
     dtgstart:   1979010100
     bounds:     time_bnds,
 <xarray.DataArray 'time' ()>
 array('2019-08-31T00:00:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time     datetime64[ns] 2019-08-31
 Attributes:
     axis:       T
     long_name:  time
     dtgstart:   1979010100
     bounds:     time_bnds)

In [12]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:36633")
client

<Client: 'tcp://127.0.0.1:36633' processes=4 threads=4, memory=32.00 GiB>

In [50]:
ds

<xarray.Dataset>
Dimensions:  (rlon: 501, rlat: 454, time: 14853, height: 1)
Coordinates:
  * rlon     (rlon) float64 -32.0 -31.95 -31.9 -31.85 ... -12.6 -12.55 -12.55
  * rlat     (rlat) float64 -8.7 -8.65 -8.6 -8.55 -8.5 ... 5.9 5.9 5.95 5.95
    lon      (rlat, rlon) float64 dask.array<chunksize=(454, 501), meta=np.ndarray>
    lat      (rlat, rlon) float64 dask.array<chunksize=(454, 501), meta=np.ndarray>
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-08-31
  * height   (height) float64 0.0
Data variables:
    t2m      (time, height, rlat, rlon) float32 dask.array<chunksize=(300, 1, 454, 501), meta=np.ndarray>

Time how long it takes to compute the mean of the dataset. I got 3m 18a on the 32GB, LEAP pangeo server, using a 20-worker dask cluster. 

In [51]:
%%time
ds.t2m.mean().compute()

CPU times: user 232 ms, sys: 10.5 ms, total: 242 ms
Wall time: 3min 18s


<xarray.DataArray 't2m' ()>
array(265.21317, dtype=float32)

Write a very small version to zarr as a test. 

In [39]:
import fsspec
import json
filename = 'gs://ldeo-glaciology/RACMO/AP_new/test_JK/t2m_small_v2'

mapper = fsspec.get_mapper(filename, mode='w',token=token)

t2m_small = ds.isel(time = slice(0,1000))
t2m_small.to_zarr(mapper)

Write the full t2m dataset to zarr in the google bucket.

In [42]:
import fsspec
import json
filename = 'gs://ldeo-glaciology/RACMO/AP_new/test_JK/t2m_all_v1'

mapper = fsspec.get_mapper(filename, mode='w',token=token)

ds.to_zarr(mapper)


Load the data from zarr to check that we can. 

In [43]:
t2m_reloaded = xr.open_dataset(filename, engine='zarr', consolidated=True, chunks={}) 

Time how long it takes to compute mean. It took 16 s on the 32GB, LEAP pangeo server using a 20-worker dask cluster.

In [52]:
%%time
t2m_reloaded.t2m.mean().load()

CPU times: user 37.4 ms, sys: 1.93 ms, total: 39.3 ms
Wall time: 15.9 s


<xarray.DataArray 't2m' ()>
array(265.21317, dtype=float32)